In [1]:
# This notebook performs data fetch for OF partial dependence plot generation

In [2]:
# Install packages (one-time)

In [ ]:
%%bash
source $VIRTUAL_ENV_DIR/python3/bin/activate

install_package_python3.sh add dsw_qr==0.1.13

$VIRTUAL_ENV_DIR/python3/bin/python -m pip install galileo
$VIRTUAL_ENV_DIR/python3/bin/python -m pip install galileo-py
$VIRTUAL_ENV_DIR/python3/bin/python -m pip install tchannel

In [ ]:
# fetch data

In [ ]:
import os

import pandas as pd
from dsw_qr import dsw_qr

In [4]:
USER_EMAIL = 'targupt@uber.com'

In [ ]:
QUERY = """select
  msg.supply_type,
  msg.adjustedeta,
  msg.eyeball_eta,
  msg.fd_eta,
  msg.unadjusted_eta,
  msg.trip_distance_meter,
  msg.trip_duration_sec,
  msg.objective_value
from
  rawdata.kafka_hp_multileg_mgv_log_nodedup
where
  datestr between '2022-10-24'
  and '2022-10-30'
  and msg.flow_type in ('solo_batch', 'solo')
  and msg.job_type = 'PERSONAL_TRANSPORT'
  and msg.calculator_type = 'compound_completion_rate_of'
  and abs(
    mod(
      from_big_endian_64(xxhash64(CAST(msg.job_uuid AS varbinary))),
      100000
    )
  ) <= 1"""

In [ ]:
from queryrunner_client import Client
qr = Client(user_email='targupt@uber.com')
qr.list_datasources()
cursor = qr.execute("hive", QUERY)
result = cursor.fetchall()

In [10]:
pd.DataFrame(result).to_csv("data/of_prediction_sample.csv", index=False)